In [1]:
import os
import re
from operator import itemgetter
from textwrap import dedent
from typing import List

import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import pandas as pd
import seaborn as sns
from datasets import load_dataset
from dotenv import load_dotenv
from edgar import Company, CompanyFiling, CompanyFilings, set_identity
from langchain.schema import Document
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from langchain_community.vectorstores import SupabaseVectorStore
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langsmith import Client as LangsmithClient
from langsmith.evaluation import LangChainStringEvaluator, evaluate
from rich import print
from supabase.client import Client, create_client

load_dotenv()
os.environ["LANGCHAIN_PROJECT"] = "llm-evals"
%load_ext rich


In [18]:
supabase_url = os.environ.get("SUPABASE_URL")
supabase_key = os.environ.get("SUPABASE_PRIVATE_KEY")
supabase: Client = create_client(supabase_url, supabase_key)

embeddings = OpenAIEmbeddings(model="text-embedding-3-small", dimensions=512)

vectorstore = SupabaseVectorStore(
    supabase,
    embedding=embeddings,
    table_name="documents_duplicate",
    query_name="match_documents_duplicate",
)


In [21]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_openai import OpenAI

metadata_field_info = [
    AttributeInfo(
        name="ticker",
        description="The ticker of the company",
        type="string or list[string]",
    ),
    AttributeInfo(
        name="year",
        description="The year of the report",
        type="integer or list[integer]",
    ),
]

document_content_description = "Chunks of text from financial reports"

llm = OpenAI(temperature=0)

retriever = SelfQueryRetriever.from_llm(
    llm, vectorstore, document_content_description, metadata_field_info, verbose=True
)


In [25]:
# This example only specifies a filter
retriever.invoke("What is COST's risk?")


[01:03:18] INFO     Generated Query: query='risk' filter=Comparison(comparator=<Comparator.EQ: 'eq'>,   ]8;id=639096;file://e:\Projects\AI-Maven\AIE3\.venv\Lib\site-packages\langchain\retrievers\self_query\base.py\base.py]8;;\:]8;id=955066;file://e:\Projects\AI-Maven\AIE3\.venv\Lib\site-packages\langchain\retrievers\self_query\base.py#267\267]8;;\
                    attribute='ticker', value='COST') limit=None                                                   


[
    Document(
        page_content='of coverage to account for these variables, actual claims and costs could differ significantly from recorded liabilities. Historically, adjustments to our estimates have not been material. Recent Accounting Pronouncements We do not expect that any recently issued accounting pronouncements will have a material effect on our financial statements.',
        metadata={'form': '10-K', 'year': 2022, 'ticker': 'COST', 'item_name': 'Item 7'}
    ),
    Document(
        page_content='or are reasonably likely to have a material current or future effect on our financial condition or financial statements. Critical Accounting Estimates The preparation of our consolidated financial statements in accordance with U.S. generally accepted accounting principles (U.S. GAAP) requires that we make estimates and assumptions that affect the reported amounts of assets and liabilities and the disclosure of contingent assets and liabilities at the date of the financial sta

In [11]:
# This example only specifies a relevant query
retriever.invoke("What are some movies about dinosaurs")


[00:55:41] INFO     Generated Query: query='dinosaurs' filter=None limit=None                           ]8;id=742024;file://e:\Projects\AI-Maven\AIE3\.venv\Lib\site-packages\langchain\retrievers\self_query\base.py\base.py]8;;\:]8;id=264648;file://e:\Projects\AI-Maven\AIE3\.venv\Lib\site-packages\langchain\retrievers\self_query\base.py#267\267]8;;\


[
    Document(
        page_content='A bunch of scientists bring back dinosaurs and mayhem breaks loose',
        metadata={'year': 1993, 'genre': 'science fiction', 'rating': 7.7}
    ),
    Document(
        page_content='Toys come alive and have a blast doing so',
        metadata={'year': 1995, 'genre': 'animated'}
    ),
    Document(
        page_content='Leo DiCaprio gets lost in a dream within a dream within a dream within a ...',
        metadata={'year': 2010, 'rating': 8.2, 'director': 'Christopher Nolan'}
    ),
    Document(
        page_content='A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea',
        metadata={'year': 2006, 'rating': 8.6, 'director': 'Satoshi Kon'}
    )
]

In [12]:
# This example specifies a query and composite filter
retriever.invoke(
    "What's a movie after 1990 but before (or on) 2005 that's all about toys, and preferably is animated"
)


[00:56:00] INFO     Generated Query: query='toys' filter=Operation(operator=<Operator.AND: 'and'>,      ]8;id=471441;file://e:\Projects\AI-Maven\AIE3\.venv\Lib\site-packages\langchain\retrievers\self_query\base.py\base.py]8;;\:]8;id=681531;file://e:\Projects\AI-Maven\AIE3\.venv\Lib\site-packages\langchain\retrievers\self_query\base.py#267\267]8;;\
                    arguments=[Operation(operator=<Operator.AND: 'and'>,                                           
                    arguments=[Comparison(comparator=<Comparator.GTE: 'gte'>, attribute='year',                    
                    value=1990), Comparison(comparator=<Comparator.LTE: 'lte'>, attribute='year',                  
                    value=2005)]), Comparison(comparator=<Comparator.LIKE: 'like'>, attribute='genre',             
                    value='animated')]) limit=None                                                                 


[
    Document(
        page_content='Toys come alive and have a blast doing so',
        metadata={'year': 1995, 'genre': 'animated'}
    )
]